In [12]:
import datetime
import random
import warnings
import pandas as pd
import numpy as np
import sys
import logging
import sqlalchemy
# import datetime 
from datetime import datetime
from dateutil.relativedelta import relativedelta
import tables
# import h5py

## import functions 
sys.path.append('/home/yet/nyc_peu_inspections/utils/') 
from load_creds_file import *
from postgres_functions import *
from savebackups_S3 import *
from choosingfeatures_functions import *
from preprocessing_functions import *
from get_feature_var_types import *
from etl_functions import *
## log file
dataname = 'test_runmodels'
log_filename = dataname + "_logfile_run.log"
logging.basicConfig(level=logging.INFO, filename = log_filename)
log = logging.getLogger(__name__)

# load creds
creds = load_creds_file('/home/yet/credentials_ty.yaml')

loaded yaml file to use


In [40]:
from sklearn.ensemble import RandomForestClassifier

### set up parameters

In [15]:
############# split dates to use 
param_train_startdate = datetime(2017,3,1).date() # pick start date for training data 
param_test_enddate = datetime(2018,2,1).date() # NOTE THIS IS A DUMMY FOR CONFIG DICTIONARY IN THIS RUN pick end date for test data
param_test_months = 1


In [8]:
param_which_timeunit = 'month' # model isn't set up to run weekly, but we could make the split function dynamic (which won't be that hard)
param_label_type = 'binary' # choose whether to use binary, continuous, or ratio 
param_primary_label = 'internal_cases_opened_any_next_month' # choose primary label to subset on for missingness 
# param_label_quantile_threshold = 0.9 # if primary label is a ratio, quantile threshold over which ratio is coded as one
param_only_observed_labels_train = True # choose whether or not to include obs with missing labels
param_test_targetzip = True # choose whether test data should include only addresses from TSU target zips 


In [17]:
############ finding top k
param_percents_ofunits = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5] # if not hard-coding top k, percentage of minimum units knocked in that borough across all months to set top k to


### Read in data

In [4]:
### set up database connection
cursor, psy_connect, alchemy_connection = startengine_alchemy(creds = creds, return_raw_connection = True)


In [24]:
### query table
train_query = '''select * from dssg_staging_anycase.testing_cleaned_train_split_20170301_features_for_deep_dive;'''
train_df = readquery_todf_postgres(query=train_query, alchemy_connection=alchemy_connection)

In [33]:
### query table
test_query = '''select * from dssg_staging_anycase.testing_cleaned_test_split_20170301_features_for_deep_dive;'''
test_df = readquery_todf_postgres(query=test_query, alchemy_connection=alchemy_connection)

In [36]:
label_train_mat = train_df[[param_primary_label]].values
label_test_mat =  test_df[[param_primary_label]].values

In [47]:
feature_list = list(train_df.columns)
feature_list.remove(param_primary_label)
feature_list.remove(['address_id'])
feature_list.remove(['month_start'])
x_train_features = train_df[feature_list]
x_test_features = test_df[feature_list]

ValueError: list.remove(x): x not in list

### fit the model

In [45]:
one_model = RandomForestClassifier()

one_model.fit(x_train_features.values, label_train_mat)

/home/yet/.pyenv/versions/3.6.7/envs/yet_pyenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


TypeError: float() argument must be a string or a number, not 'Timestamp'